## 24. Monitoring file‑system events

Real‑time watchers (Linux inotify, macOS FSEvents, Windows ReadDirectoryChangesW) notify your app when files change. In Python, use **watchdog** library for a cross‑platform wrapper—great for auto‑reload, build tools.

```python
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time, pathlib

class Handler(FileSystemEventHandler):
    def on_modified(self, event):
        print('modified', event.src_path)

path = pathlib.Path('.').resolve()
observer = Observer()
observer.schedule(Handler(), path=str(path), recursive=False)
observer.start()
print('Watching', path)
time.sleep(2); observer.stop(); observer.join()
```

### Quick check

1. watchdog relies on polling when native API missing?
  a. True   b. False

2. True / False inotify works on Windows.

<details><summary>Answer key</summary>

1. **a**.
2. **False**.

</details>

## 25. Case sensitivity & path separators across OSes

Windows file‑system is case‑insensitive by default; Linux/mac (APFS) often case‑sensitive. Use `Path` methods, not string compares. Path separator: `/` on Unix, `\` on Windows—`pathlib` handles both.

```python
from pathlib import PurePath
print(PurePath('a/b').as_posix())
print(PurePath('a\\b').parts)
```

### Quick check

1. Comparing 'Readme.md' vs 'README.MD' on Windows returns equal?
  a. yes   b. no

2. True / False Using forward slashes in Python paths on Windows usually works.

<details><summary>Answer key</summary>

1. **a**.
2. **True** – Windows API accepts '/'.

</details>

## 26. Unicode filenames and cross‑platform pitfalls

Filenames are bytes on Unix, UTF‑16 on Windows. Python decodes to `str` using filesystem encoding (`utf‑8` on modern distros). Always store paths as `Path` objects; beware of normalisation (NFC vs NFD) when exchanging between macOS and Linux.

```python
from pathlib import Path
name = 'żółć.txt'
Path(name).write_text('hi', encoding='utf-8')
print(list(Path('.').glob('*.txt')))
```

### Quick check

1. macOS normalises filenames to:
  a. NFC   b. NFD

2. True / False Bytes path APIs (`os.listdir(b'.')`) bypass decoding.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 27. Filesystem timestamps (`atime`, `mtime`, `ctime`)

`stat()` returns:  
* **st_atime** – last access; often disabled for performance.  
* **st_mtime** – last modification.  
* **st_ctime** – metadata change (Unix) or creation time (Windows).

```python
import os, time, pathlib
p = pathlib.Path('pi.txt')
stat = p.stat()
print(time.ctime(stat.st_mtime))
```

### Quick check

1. `ctime` on Windows means:
  a. metadata change   b. creation

2. True / False Touching file updates mtime.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – unless using utime keep flag.

</details>

## 28. Sparse files and holes

Sparse files reserve virtually large size but physical blocks allocated on write. Databases and VM images use them. Reading unwritten region yields zeros. `fallocate` on Linux or `SetFileValidData` on Windows creates holes.

```python
with open('sparse.bin','wb') as f:
    f.seek(10**9)  # 1 GB hole
    f.write(b'X')
```

### Quick check

1. Disk usage (`du`) shows small size for sparse?
  a. True   b. False

2. True / False Copying sparse file with `cp` always preserves holes.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – need `--sparse`.

</details>

## 29. Quotas, limits, and handling “disk full”

Always catch `OSError` `ENOSPC` on writes; application should clean temp or alert user. Check free space with `shutil.disk_usage`. Quotas may hit before physical full.

```python
import shutil, pathlib
total, used, free = shutil.disk_usage(pathlib.Path('.'))
print('free MB', free//1_000_000)
```

### Quick check

1. `ENOSPC` stands for:
  a. No space left   b. No such file

2. True / False Disk quota errors raise same errno as disk full.

<details><summary>Answer key</summary>

1. **a**.
2. **True** – both ENOSPC.

</details>

## 30. Atomic file writes (temp + rename)

Write to temp file in same dir → `flush`+`fsync` → `os.replace(temp, target)`; rename is atomic on same filesystem, preventing readers from seeing half-written data.

```python
import os, tempfile, pathlib
target = pathlib.Path('config.json')
with tempfile.NamedTemporaryFile('w', dir=target.parent, delete=False) as tmp:
    tmp.write('{}')
    tmp.flush(); os.fsync(tmp.fileno())
os.replace(tmp.name, target)
```

### Quick check

1. Atomic rename fails across:
  a. same FS   b. different FS

2. True / False `os.replace` overwrites existing file.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>